In [5]:
!pip install -q \
    transformers==4.31.0 \
    accelerate==0.21.0 \
    bitsandbytes==0.41.0 \
    sentence-transformers==2.2.2 \
    xformers==0.0.20 \

# !pip install -q \
#     langchain==0.1.0 \
#     langchain-community==0.0.12 \
#     langchainhub==0.1.14 \
#     faiss-cpu
#     # faiss-gpu \

# !pip install -U langsmith
# !pip install -q pandas
# # !pip install -q colab-xterm
# !pip install -qU langchain-anthropic
# !pip install -q python-dotenv


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough


from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders.dataframe import DataFrameLoader
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_anthropic import ChatAnthropic
from langchain_cohere import ChatCohere



# from transformers import pipeline
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import huggingface_hub as hf_hub
# import torch

try:
    from google.colab.userdata import get as getenv
    print("Running in colab")
except ImportError:
    from os import getenv
    import dotenv
    dotenv.load_dotenv()

In [2]:
try:
    from torch import cuda
    device = 'cuda' if cuda.is_available() else 'cpu'
except ImportError:
    device = 'cpu'

print("Device:", device)

Device: cpu


In [3]:
HF_TOKEN = getenv('HF_TOKEN')
assert HF_TOKEN, "A valid HuggingFace token is required to be set as <HF_TOKEN>."
hf_hub.login(HF_TOKEN)

c:\Users\mehra\mehran\dltorch\RestoRecommender\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\mehra\.cache\huggingface\token
Login successful


In [4]:
ANTHROPIC_API_KEY = getenv('ANTHROPIC_API_KEY')
COHERE_API_KEY = getenv('COHERE_API_KEY')

## Set up LangSmith

In [5]:
LANGCHAIN_API_KEY = getenv('LANGCHAIN_API_KEY')
LANGCHAIN_ENDPOINT = getenv('LANGCHAIN_ENDPOINT')
assert LANGCHAIN_API_KEY, "An API key for LangChainSmith is required to be set as <LANGCHAIN_API_KEY>."

In [5]:
OPENAI_API_KEY = getenv('OPENAI_API_KEY')

## Constants

In [5]:
# Dataset files
PLACES_PATH = "data/places.csv"
REVIEWS_PATH = "data/reviews.csv"

## Models
# LLM_MODEL format: <model_type>::<model_name>
# 
# model types: ['hf', 'anthropic']
# 
# Example:
# hf::meta-llama/Llama-2-7b-hf
# anthropic::claude-3-sonnet-20240229
# 
# LLM_MODEL = "anthropic::claude-3-sonnet-20240229"
LLM_MODEL = "anthropic::claude-3-opus-20240229"
LLM_MODEL = "cohere::"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"

# Embeddings
EMBEDDINGS_CACHE_STORE="./cache/"

# Faiss
FAISS_REVIEWS_PATH = "faiss_index"
FAISS_INDEX_NAME = "index"
FAISS_DISTANCE_STRATEGY='EUCLIDEAN_DISTANCE'

## Load Dataset

Here we are using 2 csv files containing places (restuarants, bars, ...) info and reviews for each of them.

In [6]:
def get_documents(content_func=lambda row:row['review'],
                  source_func=lambda row:row['place_id'],
                  metadata_fields=[]):

  # Load both data files
  places_df = pd.read_csv(PLACES_PATH)
  reviews_df = pd.read_csv(REVIEWS_PATH)
  reviews_df.drop_duplicates(inplace=True)
  places_df.drop_duplicates(inplace=True)

  # merge them on 'place_id'
  merged_df = pd.merge(places_df, reviews_df, on='place_id', how='inner')

  # add page_content and source columns using their corresponing functions
  merged_df['page_content'] = merged_df.apply(content_func, axis=1)
  merged_df['source'] = merged_df.apply(source_func, axis=1)

  # update metadata_fields with 'page_content', 'source'
  metadata_fields = list(set(metadata_fields + ['page_content', 'source']))

  loader = DataFrameLoader(merged_df[metadata_fields],page_content_column='page_content')
  return loader.load()

In [7]:
# def content_func(row) -> str:
#   content_fields = ['place_name', 'place_types', 'place_address', 'place_average_ratings', 'review']
#   return '\n'.join(f"{key}={row[key]}" for key in content_fields)

def content_func(row) -> str:
  content_fields = ["place_name",
                    "place_address",
                    "place_average_ratings",
                    "place_website",
                    "place_phone_number",
                    "place_price_level",
                    "place_primary_type",
                    "has_delivery",
                    "is_reservable",
                    "places_opening_hours",
                    "review",
                    ]
  return '\n'.join(f"{key}={row[key]}" for key in content_fields)

documents = get_documents(content_func)

In [8]:
documents[0]

Document(page_content='place_name=Roemerkeller Padova\nplace_address=Via Romana Aponense, 137, 35142 Padova PD, Italy\nplace_average_ratings=4.0\nplace_website=http://www.roemerkeller.it/\nplace_phone_number=+39 328 178 6333\nplace_price_level=PRICE_LEVEL_MODERATE\nplace_primary_type=italian_restaurant\nhas_delivery=False\nis_reservable=True\nplaces_opening_hours=[\'Monday: 12:00\\u2009–\\u20092:30\\u202fPM, 6:30\\u202fPM\\u2009–\\u200912:00\\u202fAM\', \'Tuesday: 12:00\\u2009–\\u20092:30\\u202fPM, 6:30\\u202fPM\\u2009–\\u200912:00\\u202fAM\', \'Wednesday: 12:00\\u2009–\\u20092:30\\u202fPM, 6:30\\u202fPM\\u2009–\\u200912:00\\u202fAM\', \'Thursday: 12:00\\u2009–\\u20092:30\\u202fPM, 6:30\\u202fPM\\u2009–\\u200912:00\\u202fAM\', \'Friday: 12:00\\u2009–\\u20092:30\\u202fPM, 6:30\\u202fPM\\u2009–\\u200912:00\\u202fAM\', \'Saturday: 6:30\\u202fPM\\u2009–\\u200912:30\\u202fAM\', \'Sunday: 12:00\\u2009–\\u20093:00\\u202fPM, 6:30\\u202fPM\\u2009–\\u200912:00\\u202fAM\']\nreview=Huge "eating pa

## Load Embeddings model

In [21]:
def get_hf_embedding_model(embedding_model_name,
                           cache_embeddings_store,
                           device='cpu',
                           normalize_embeddings=False,
                           ):
  model_kwargs = {'device': device, 
                  'trust_remote_code': True
                  }
  encode_kwargs = {'normalize_embeddings': normalize_embeddings} # Set `True` for cosine similarity
  embedding_model = HuggingFaceEmbeddings(
      model_name=embedding_model_name,
      model_kwargs=model_kwargs,
      encode_kwargs=encode_kwargs
      )
  store = LocalFileStore(cache_embeddings_store)
  embedding_model = CacheBackedEmbeddings.from_bytes_store(
                    embedding_model, store)
  return embedding_model

In [12]:
# embedding_model = get_hf_embedding_model(EMBEDDING_MODEL_NAME,
#                                          EMBEDDINGS_CACHE_STORE,
#                                          device=device,
#                                          normalize_embeddings=False)

c:\Users\mehra\mehran\dltorch\RestoRecommender\venv\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [22]:
# pip install -U sentence-transformers==2.4.0
#  "jinaai/jina-embeddings-v2-base-en"
EMBEDDING_MODEL_NAME = 'Alibaba-NLP/gte-large-en-v1.5'
embedding_model = get_hf_embedding_model(EMBEDDING_MODEL_NAME,
                                         EMBEDDINGS_CACHE_STORE,
                                         device=device,
                                         normalize_embeddings=False)

In [23]:
import numpy as np
test = np.array([len(doc.page_content) for doc in documents])
test
test[0], len(test), test.mean(), np.percentile(test, 99.5), test.argmax(),

# print(documents[304].page_content)

(1041, 585, 976.6940170940171, 2168.4000000000083, 304)

In [24]:
2000 * 2 * 585 

2340000

## Load FAISS (Vector Database)

In [11]:
def get_vector_database(documents, embedding_model):

  vector_database = FAISS.from_documents(
      documents, embedding_model,
      distance_strategy= FAISS_DISTANCE_STRATEGY
      
      )
  return vector_database

In [12]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore

In [13]:
# vector_db = get_vector_database(documents, embedding_model)

embedding_size = 8192
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embedding_model, index, InMemoryDocstore({}), {})

In [14]:
vectorstore = vectorstore.from_documents(documents, embedding_model,
                            distance_strategy= FAISS_DISTANCE_STRATEGY)

In [18]:
vectorstore.similarity_search("which one is the best pizza restaurant in the city?", k = 1)

AssertionError: 

In [15]:
vectorstore.save_local(folder_path=FAISS_REVIEWS_PATH, index_name=FAISS_INDEX_NAME)

In [26]:
## if you want to save the db and use the files to load it again later.
FAISS_REVIEWS_PATH_EUCLIDEAN = "faiss_index_euclidean"
FAISS_REVIEWS_PATH = FAISS_REVIEWS_PATH_EUCLIDEAN
vector_db.save_local(folder_path=FAISS_REVIEWS_PATH, index_name=FAISS_INDEX_NAME)
vector_db = FAISS.load_local(folder_path=FAISS_REVIEWS_PATH,
                             embeddings=embedding_model,
                             index_name=FAISS_INDEX_NAME)

In [17]:
docs = vectorstore.similarity_search("which one is the best pizza restaurant in the city?", k = 5)

AssertionError: 

In [29]:
docs[0]

Document(page_content="place_name=La Scuderia\nplace_address=Via San Lorenzo, 13, 35031 Abano Terme PD, Italy\nplace_average_ratings=4.0\nplace_website=http://www.lascuderiahb.it/\nplace_phone_number=+39 393 797 8770\nplace_price_level=PRICE_LEVEL_MODERATE\nplace_primary_type=pizza_restaurant\nhas_delivery=True\nis_reservable=True\nplaces_opening_hours=['Monday: 6:30\\u202fPM\\u2009–\\u20092:00\\u202fAM', 'Tuesday: Closed', 'Wednesday: 6:30\\u202fPM\\u2009–\\u200912:00\\u202fAM', 'Thursday: 6:30\\u202fPM\\u2009–\\u20092:00\\u202fAM', 'Friday: 6:30\\u202fPM\\u2009–\\u20092:00\\u202fAM', 'Saturday: 6:30\\u202fPM\\u2009–\\u20092:00\\u202fAM', 'Sunday: 12:00\\u202fPM\\u2009–\\u20092:00\\u202fAM']\nreview=Excellent pizza served in a pleasant environment, modern and classic. Large selection of dishes, fast service", metadata={'source': 'ChIJrzxAcjTZfkcR-_oZ1mU6F9I'})

In [20]:
question = "where is the Enoteca Barcollo located? and what is its phone number?"

docs = vector_db.similarity_search(question, k = 5)

for i in range(5):
  print(docs[i], end="\n\n")

page_content='place_name=Enoteca Barcollo\nplace_address=Via Alessandro Guidi, 23, 35142 Padova PD, Italy\nplace_average_ratings=4.3\nplace_website=nan\nplace_phone_number=+39 393 082 6866\nreview=Top' metadata={'source': 'ChIJwzs5vKLbfkcRp43hqXskPKw'}

page_content='place_name=Enoteca Barcollo\nplace_address=Via Alessandro Guidi, 23, 35142 Padova PD, Italy\nplace_average_ratings=4.3\nplace_website=nan\nplace_phone_number=+39 393 082 6866\nreview=Brave!' metadata={'source': 'ChIJwzs5vKLbfkcRp43hqXskPKw'}

page_content='place_name=Enoteca Barcollo\nplace_address=Via Alessandro Guidi, 23, 35142 Padova PD, Italy\nplace_average_ratings=4.3\nplace_website=nan\nplace_phone_number=+39 393 082 6866\nreview=Super cicchetti, equally super cold cuts platter! Wide choice of wines. Very welcoming environment and super friendly staff. Both the boy and the girl made themselves available to serve us as best as possible and guarantee us an excellent aperitif! 💪✨💯💯💯💯' metadata={'source': 'ChIJwzs5vKLbfk

In [21]:
question = "where is la mafaldina?"

docs = vector_db.similarity_search(question, k = 5)

for i in range(5):
  print(docs[i], end="\n\n")

page_content="place_name=La Mafaldina Pizzeria - Pontevigodarzere\nplace_address=Via dei Vivarini, 22, 35133 Padova PD, Italy\nplace_average_ratings=4.5\nplace_website=https://www.lamafaldina.it/\nplace_phone_number=+39 049 735 8018\nreview=Me and my family went there to have dinner, based on the reviews that I red about this place and it was defenetely a very good choice, because pizza is just great and, if you are in the neighborhood, you don't want to miss it." metadata={'source': 'ChIJWdpV5_7RfkcRI93ap-aXK90'}

page_content="place_name=La Mafaldina Pizzeria - Pontevigodarzere\nplace_address=Via dei Vivarini, 22, 35133 Padova PD, Italy\nplace_average_ratings=4.5\nplace_website=https://www.lamafaldina.it/\nplace_phone_number=+39 049 735 8018\nreview=A very beautiful place with amazing and helpful staff. It's one of rhe best pizza restaurants in Padua. 💕 Grazie!" metadata={'source': 'ChIJWdpV5_7RfkcRI93ap-aXK90'}

page_content='place_name=La Mafaldina Pizzeria - Pontevigodarzere\nplac

## Load LLM

In [36]:
def get_anthropic_api_llm(model_name):
  llm = ChatAnthropic(model_name=model_name, anthropic_api_key=ANTHROPIC_API_KEY,)

  return llm

def get_cohere_api_llm():
  llm = ChatCohere()

  return llm


In [14]:
def get_hf_llm(model_name):

  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
  model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, )
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  pipe = pipeline(
      model=model,
      tokenizer=tokenizer,
      return_full_text=True,  # langchain expects the full text
      task='text-generation',
      # we pass model parameters here too
      temperature=0.0001,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
      max_new_tokens=512,  # mex number of tokens to generate in the output
      repetition_penalty=1.1  # without this output begins repeating
  )

  llm = HuggingFacePipeline(pipeline=pipe,)
  return llm


In [37]:
model_type, _, model_name = LLM_MODEL.partition('::')

# If model type is not set, use anthropic
if model_name == "":
    model_type = "cohere"
    model_name = model_type


if model_type == "anthropic":
    llm = get_anthropic_api_llm(model_name)
elif model_type == "hf":
    llm = get_hf_llm(model_name)
else:
    llm = get_cohere_api_llm()
    
llm

ChatCohere(client=<cohere.client.Client object at 0x0000024185F73A60>, async_client=<cohere.client.AsyncClient object at 0x0000024185AA6200>, cohere_api_key=SecretStr('**********'))

In [13]:
llm.invoke("Hi")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '7614c8a9-de6f-445b-860f-fe487b0afa16', 'token_count': {'input_tokens': 71, 'output_tokens': 9}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '7614c8a9-de6f-445b-860f-fe487b0afa16', 'token_count': {'input_tokens': 71, 'output_tokens': 9}}, id='run-9a768896-7db1-46d9-bad3-7dc27af2aa1c-0')

In [24]:
llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'id': 'msg_017VUzakc4UbQijWgNc2ieFe', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 8, 'output_tokens': 12}}, id='run-9b6816f6-b5df-4949-87c2-e9ef1577ea7b-0')

## Create LangChain pipeline

In [38]:
review_template_str = """
Your job is to use Google Map restaurants and bars reviews to help people find best places to go for a meal or a drink.
Use the following information and reviews to answer the questions.
If you don't know an answer based on the context, say you don't know. Answer context:
{context}
"""
## """
# If you don't know an answer based on the context, say you don't know, and
# if the context is not about restaurants, then kindly tell them that  you can
# only provide assistance and answer questions related to restaurants.
##"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [review_system_prompt, review_human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

FAISS_REVIEWS_PATH_EUCLIDEAN = "faiss_index_euclidean"
FAISS_REVIEWS_PATH = FAISS_REVIEWS_PATH_EUCLIDEAN
vector_db = FAISS.load_local(folder_path=FAISS_REVIEWS_PATH,
                             embeddings=embedding_model,
                             index_name=FAISS_INDEX_NAME)
reviews_retriever = vector_db.as_retriever(search_kwargs={'k': 20,
                                                        #   'fetch_k': 50,
                                                          })

review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | llm
    | StrOutputParser()
)

In [39]:
question = """Where can I find delicious pizzas?"""
print(review_chain.invoke(question))

AssertionError: 

In [19]:
question = """Where can I find delicious pizzas?"""
print(review_chain.invoke(question))

Based on the reviews, several places in Padua, Italy are mentioned as having great pizza:

1. Enoteca Barcollo - The review says they have "THE BEST PIZZA I HAVE TASTED IN RECENT YEARS" with quality ingredients and perfectly balanced crust. 

2. Fastdrink24 - One review says "If you are looking for a place to eat good pizza, search no more. This is the right place to be." However, another calls it "The worst pizza worldwide", so opinions seem mixed.

3. iDon Padova - A review states it has "One of the best pizza I had" and another recommends to "Go for a pizza".

4. Ristorante Napoli Centrale - The pizza here is described as "Magnificent" and the review mentions the meal was very tasty.

5. Villa Italia - A review claims they had dinner here and "it is the best pizza I've had in 5 years."

So in summary, Enoteca Barcollo, iDon Padova, Ristorante Napoli Centrale and Villa Italia seem to be top choices for delicious pizza in Padua based on these reviews, with Fastdrink24 getting mixed op

In [31]:
question = """What are the pros and cons of the best pizza restaurant in the city?"""
print(review_chain.invoke(question))

# {"context": reviews_retriever, "question": RunnablePassthrough()}

# review_prompt_template

Based on the reviews, Enoteca Barcollo seems to have the best pizza in Padua. Here are the pros and cons mentioned:

Pros:
- Has the best pizza the reviewer has tasted in recent years
- Quality ingredients used for the pizza toppings
- Wholemeal pizza dough that is perfectly balanced in terms of crunchiness, thickness and flavor
- Even the reviewer's girlfriend, who normally doesn't like pizza, almost wanted to order a second one

Cons:
- The pizza is not large in size 
- It costs €9, which the reviewer feels is worth it for the quality, but some may find a bit pricey

Another strong contender for great pizza is La Mafaldina Pizzeria. The review praises its excellent pizzas with outstanding quality ingredients, calling it one of the best pizzerias in Padua. The only con mentioned is the limited topping choices for customizing pizzas.


In [65]:
test = review_prompt_template.invoke({"context": reviews_retriever.invoke(question), "question": question})
print(test)
# StrOutputParser()
print(test.messages[0].content)

test.messages[1].content

messages=[SystemMessage(content='\nYour job is to use Google Map restaurants and bars reviews to help people find best places to go for a meal or a drink.\nUse the following information and reviews to answer the questions.\nIf you don\'t know an answer based on the context, say you don\'t know. Answer context:\n[Document(page_content=\'place_name=Soul Kitchen\\nplace_address=Via del Santo, 23, 35123 Padova PD, Italy\\nplace_average_ratings=4.6\\nplace_location=nan\\nplace_website=nan\\nplace_phone_number=+39 049 664818\\nreview=I got a tip to eat here from a local as my need was to eat food fast.\\r\\n\\r\\nThe glass of wine was nice, the burger was fair in quality. I tried the carbonara burger. Taste experience-wise, I was not too impressed and I feel the price for the offer was on the high end (14 euro) for not including fries.\\r\\n\\r\\nThe staff was helpful and the place has a nice atmosphere.\', metadata={\'source\': \'ChIJP-ZEOlTafkcRFi8zBVerMvg\'}), Document(page_content="place

'What are the pros and cons of the best restaurant in the city?'

In [87]:
question = """What are the pros and cons of the best restaurant in the city?"""

print(review_chain.invoke(question))

Based on the reviews provided, there is not enough information to definitively determine the single best restaurant in Padova. The reviews cover several different restaurants, each with some positive aspects mentioned:

Soul Kitchen - Nice atmosphere and helpful staff, but the reviewer felt the burger was overpriced for the quality.

Bistro Mexicano - Friendly employees, large portions and delicious food, but the upstairs seating area is a bit small. 

Exforo - Great view of Prato della Valle, kind service, but cocktails and food are overpriced.

Hamerica's - Cozy atmosphere, tasty burgers, 50% off beer special, lovely waiter.

Gourmetteria - High quality, modern takes on traditional Veneto dishes, good for vegetarians, open kitchen, good wine selection, informal and reasonably priced. The duck is recommended. Reviewer goes frequently with family.

Of these, Gourmetteria has the most detailed positive review, so it could potentially be considered the best of this set. But there isn't e

In [66]:
question = """Give the name, address and phone number of the best steak houses?"""

print(review_chain.invoke(question))

Based on the reviews provided, the best steak house appears to be Ristorante Vecchio Falconiere. Here are the details:

Name: Ristorante Vecchio Falconiere
Address: Via Umberto I, 31, 35122 Padova PD, Italy  
Phone Number: +39 049 656544

The review mentions they have a large variety of steaks from local to Wagyu beef in different sizes. The owner cooks the steak tableside and they have a large wine selection as well. The reviewer recommends making a reservation in advance as it can get fully booked.


In [67]:
question = """Give the name, address and phone number of the best steak houses with a 50 euro budget?"""

print(review_chain.invoke(question))

Based on the reviews provided, the best steak house option with a 50 euro budget seems to be Ristorante Vecchio Falconiere. Here are the details:

Ristorante Vecchio Falconiere
Address: Via Umberto I, 31, 35122 Padova PD, Italy 
Phone: +39 049 656544

The review mentions they have a large variety of steaks in different sizes, from local to Wagyu beef. It's described as a "gem for meat and wine lovers". However, the review doesn't specify prices, so I can't say for certain if it would fit a 50 euro per person budget. The other restaurants reviewed don't appear to specialize in steak.


In [26]:
question = """Give the name, address and phone number of the some good sandwich places?"""

print(review_chain.invoke(question))

Based on the reviews, a couple good sandwich places are:

Soul Kitchen
Via del Santo, 23, 35123 Padova PD, Italy
+39 049 664818

The reviews mention they have great burgers, including an Angus beef burger with excellent onion rings. One review wasn't as impressed with their carbonara burger though.

Spiller | Padova 
Via Fornace Morandi, 24/B, 35133 Padova PD, Italy
+39 049 864 4822

A review recommends their club sandwich and says the location is nice with outdoor seating available. They suggest booking in advance as it can get busy.

The other places focus more on burgers rather than a variety of sandwiches. I don't have enough information to confidently recommend additional sandwich spots beyond those two based on the reviews provided.


In [72]:
question = """How can I make a roast beef sandwich at home?"""

print(review_chain.invoke(question))

I apologize, but I did not find information about how to make a roast beef sandwich at home in the provided reviews. The reviews are about various restaurants in Padova, Italy, including Hamerica's, Autogrill Limenella, Old England Pub, Ristorante Vecchio Falconiere, Brunch Republic Padova, Bar Nazionale, and McDonald's Padova Ovest. They mention dishes like burgers, chicken burrito, vegan pulled pork sandwich, steak, and tramezzino sandwiches, but do not provide instructions for making a roast beef sandwich at home.


In [73]:
question = """What is Natural Language Processing?"""

print(review_chain.invoke(question))

I do not have information about Natural Language Processing in the provided context. The reviews are about various restaurants and bars in Italy, and do not mention anything related to Natural Language Processing.


In [36]:
question = """What are the most affordable but high-quality restaurants in City?"""
result = review_chain.invoke(question)

In [37]:
result.content
print(result.content)

Based on the reviews provided, the most affordable yet high-quality restaurant in Padova seems to be Peace 'n' Spice. A few key points from the reviews:

- It offers a variety of delicious Middle Eastern dishes at an inexpensive price level. One reviewer said "Food was amazing. Very delicious, every dish we took was very tasty."

- The ambiance and decor are described as fantastic. One review mentioned "The ambiance is fantastic and even music in background brings nostalgia."

- Service is friendly and accommodating. A reviewer said "Staffs are very friendly and polite."

- It has an average rating of 4.5 stars, higher than the McDonald's and KFC locations mentioned which are in the 3-4 star range.

The McDonald's and KFC have mixed reviews, with some praising the taste/quality but others feeling it has declined. They are also inexpensive options.

So in summary, for an affordable restaurant in Padova with high quality food, friendly service and nice atmosphere, Peace 'n' Spice stands 

## Play ground

In [99]:
for i in range(len(documents)):
    review = documents[i].page_content.split("review=")[1]
    if "The burgers are amazing," in review:
        print(review)
        print(i)

The burgers are amazing, make sure you have a green pass if you plan to eat in.
252
The burgers are amazing, make sure you have a green pass if you plan to eat in.
257
The burgers are amazing, make sure you have a green pass if you plan to eat in.
742
The burgers are amazing, make sure you have a green pass if you plan to eat in.
747


In [100]:
documents[252]

Document(page_content='place_name=Fastdrink24\nplace_address=Via Sorio, 64/A, 35141 Padova PD, Italy\nplace_average_ratings=4.2\nplace_location=nan\nplace_website=http://www.fastdrink24.it/\nplace_phone_number=+39 345 173 7833\nreview=The burgers are amazing, make sure you have a green pass if you plan to eat in.', metadata={'source': 'ChIJQR9o0zTafkcRA76TSnrQuiw'})